In [35]:
# from geopy.geocoders import Nominatim
from geopy import *
import geopy.distance
import pandas as pd
import numpy as np
import math
import itertools
import re
import Levenshtein

In [36]:
# TODO: what about voltage level?
# TODO: somehow map statuses to some sort of enum
# TODO: map column names to fixed names (see example csv)

wanted_columns_2020 = ['Investment number',
                  'This investment belongs to project number…',
                  'Commissioning Year',
                  'Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction',
                  'Type of Element',
                  'Substation From',
                  'Substation To',
                  'Technology',
                  'Total route length (km)']
params_2020 = {
    'excel': '2020/transmission.xlsx',
    'sheet': 'Trans.Investments',
    'wanted_columns': wanted_columns_2020,
    'status_column': 'Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction'
}

wanted_columns_2018 = []
params_2018 = {
    'excel': r'TYNDP_2018_Project_List.xlsx',
    'sheet': 'TYNDP 2018 Projects',
    'wanted_columns': wanted_columns_2018,
    'status_column': 'Status'
}

In [37]:
params = params_2020

In [38]:
excel = params['excel']
sheet = params['sheet']

wanted_columns = params['wanted_columns']
status_column  = params['status_column']

# TODO: header index into params
df = pd.read_excel(excel, sheet_name=sheet, header=[1])
df[wanted_columns]

,Investment number,This investment belongs to project number…,Commissioning Year,"Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction",Type of Element,Substation From,Substation To,Technology,Total route length (km)
0,2,1,2024,2,ACTransmissionLine,Pedralva (PT),Sobrado (PT),AC,67.00
1,4,1,2022,3,ACTransmissionLine,V.Minho (by Ribeira de Pena),Feira (by Ribeira de Pena),AC,131.00
2,474,1,2021,3,OnshoreSubstation,Ribeira de Pena (PT),-,AC,0.00
3,18,4,2022,3,ACTransmissionLine,Beariz (ES),Fontefria (ES),AC,30.00
4,496,4,2022,3,ACTransmissionLine,Fontefria (ES),Vila Nova de Famalicão (PT) (By Ponte de Lima),AC,140.21
...,...,...,...,...,...,...,...,...,...
316,1747,1077,2036,1,DCTransmissionLine,Thessaloniki,Dubrovo,DC,110.00
317,1748,1077,2036,1,DCTransmissionLine,Dubrovo,Ch. Mogila,DC,170.00
318,1745,1081,2036,1,OffshoreDCTransmissionCable,Tobruk,Arachtos,DC,1070.00
319,1749,1081,2036,1,DCTransmissionLine,Arachtos,Elbasan,DC,180.00


In [39]:
wanted    = df[wanted_columns]
# only choose those in permitting or under construction
wanted    = wanted.loc[wanted[status_column].astype(int) >= 3]

ac_lines  = wanted.query("`Type of Element` == 'ACTransmissionLine'")
dc_lines  = wanted.query("`Type of Element` == 'DCTransmissionLine'")
on_subst  = wanted.query("`Type of Element` == 'OnshoreSubstation'")
off_subst = wanted.query("`Type of Element` == 'OffshoreSubstation'")

lines = ac_lines.append(dc_lines)

In [5]:
# For the time being, remove lines connected to new substations
new_subst = set(on_subst['Substation From']).union(on_subst['Substation To'])
lines  = lines.query("`Substation From` not in @new_subst")
lines  = lines.query("`Substation To` not in @new_subst")

# Use bus names from buses.csv (v0.1.0)
See https://github.com/PyPSA/pypsa-eur/blob/v0.1.0rc/data/entsoegridkit/buses.csv. Data is from 2017 (newer gridkit extracts do not contain 'tags' with substation names).

In [6]:
buses_file = 'buses_v0.1.0.csv'

# see base_network.py in PyPSA-Eur repository
buses = (pd.read_csv(buses_file, quotechar="'",
                     true_values='t', false_values='f',
                     dtype=dict(bus_id="str"))
        .set_index("bus_id")
        .drop(['station_id'], axis=1)
        .rename(columns=dict(voltage='v_nom')))

In [7]:
buses = buses.query('tags.notnull()', engine='python')
buses = buses.query("symbol == 'Substation'")

# Extract 'name_eng' and 'country' from tags in  buses

In [8]:
split_regex = r'("\w+"=>"[^"]*"),' # Form: 'key => value, key => value, ...'

tag_regex   = r'"(?P<key>\w+)"=>"(?P<value>[^"]*)"' # Form: 'key => value'
tag_pattern = re.compile(tag_regex)

rows = []

for index, row in buses.iterrows():
    name    = ''
    country = ''
    x = row['x']
    y = row['y']
    
    tags_string = row['tags']
    
    tags = re.split(split_regex, tags_string)
    
    # Remove whitespaces at front and end, remove None values
    tags = [s.strip() for s in tags]
    tags = list(filter(None, tags))
    
    for tag in tags:
        m = tag_pattern.match(tag)
            
        if m is None:
            print(tag)
            
        # see group names in tag_regex
        key   = m.group('key')
        value = m.group('value')
        
        if key == 'name_eng':
            name = value.strip()
        elif key == 'country':
            country = value.strip()
    
    if name == 'unknown' or not name:
        continue
        
    rows.append((name, country, x, y))

In [9]:
curated_buses = pd.DataFrame.from_records(rows, columns=['name', 'country', 'x', 'y'])

## Remove duplicate rows

In [10]:
curated_buses = curated_buses.loc[~curated_buses.duplicated()]

## There are substations which share the same name but have different coordinates
- large deviation between coordinates => substations are most likely in different countries 
    - BUT: it does occur that different places in the same country get the same name
- small deviation between coordinates => reference to same substation (error in gridextract?)

In [11]:
# TODO: added 'NI' although Northern Ireland probably appears in PyPSA as 'GB'. Find a better solution.
pypsa_countries = ['AL', 'AT', 'BA', 'BE', 'BG', 'CH', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB', 'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'ME', 'MK', 'NI', 'NL', 'NO', 'PL', 'PT', 'RO', 'RS', 'SE', 'SI', 'SK']

### List of all duplicates

In [12]:
duplicated = curated_buses.loc[curated_buses.name.duplicated()]
duplicated = duplicated.query("country in @pypsa_countries")

# for name in duplicated.name.unique():
#     print(name)
#     for index, row in curated_buses.query('name == @name').iterrows():
#         print(f"({row['x']}, {row['y']}), {row['country']}")
#     print('----')

### Same name and country, large deviations

In [13]:
curated_buses.query("name == 'Yuzhnaya'")

,name,country,x,y
2720,Yuzhnaya,RU,44.817352,48.155093
3851,Yuzhnaya,RU,50.674438,52.002638
3905,Yuzhnaya,RU,36.268616,51.642737
3927,Yuzhnaya,RU,38.685608,51.843414
5378,Yuzhnaya,RU,59.824677,56.576128


### Same name, different country, large deviation

In [14]:
curated_buses.query("name == 'Saida'")

,name,country,x,y
617,Saida,LB,35.400696,33.587167
833,Saida,DZ,0.146942,34.908458


In [15]:
curated_buses.query("name == 'Titan'")

,name,country,x,y
1986,Titan,AL,19.786377,41.619549
2825,Titan,UA,33.767853,46.195993
5825,Titan,RU,34.026031,67.451763


## (TODO) Add new substations

In [16]:
on_subst

# extract country if it matches regex
# otherwise, np.NAN

,Investment number,This investment belongs to project number…,Commissioning Year,"Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction",Type of Element,Substation From,Substation To,Technology,Total route length (km)
2,474,1,2021,3,OnshoreSubstation,Ribeira de Pena (PT),-,AC,0.0
6,499,4,2022,3,OnshoreSubstation,Beariz (ES),Beariz (ES),AC,0.0
7,500,4,2022,3,OnshoreSubstation,Ponte de Lima (PT),Ponte de Lima (PT),AC,0.0
58,715,138,2023,4,OnshoreSubstation,Stalpu (RO),Stalpu (RO),AC,0.0
67,701,144,2025,4,OnshoreSubstation,Resita (RO),Resita (RO),AC,0.0
68,705,144,2025,3,OnshoreSubstation,Timisoara (RO),Timisoara (RO),AC,0.0
109,1711,200,2023,4,OnshoreSubstation,Kocin (CZ),NaN,AC,0.0
110,1712,200,2021,4,OnshoreSubstation,Vitkov (CZ),NaN,AC,0.0
122,631,227,2024,3,OnshoreSubstation,Bajina Basta (RS),Bajina Basta (RS),AC,0.0
125,1528,227,2020,4,OnshoreSubstation,SS Kraljevo,SS Kraljevo,AC,0.0


## Remove '(\<Country Code\>) ' from tyndp substation name strings, add new column instead
Otherwise, this could negatively impact the Levenshtein distance.

In [17]:
subst_regex   = r'(?P<place>.+)\s?[\[(](?P<country>\w{2})[)\]]' # Form: 'Glorenza (IT)'
subst_pattern = re.compile(subst_regex)

# use this if other pattern does not match to remove comments in parentheses
# e.g. 'Molai (through Sklavouna Terminal)'
alt_regex   = r'(?P<place>.+)\s?[\[(].*[)\]]'
alt_pattern = re.compile(alt_regex)

fr_names     = []
fr_countries = []
to_names     = []
to_countries = []

for index, row in lines.iterrows():    
    fr = row['Substation From']
    to = row['Substation To']
    
    # default values if regex does not match
    fr_name = fr
    to_name = to    
    fr_country = np.NAN
    to_country = np.NAN
    
    fr_match = subst_pattern.match(fr)
    to_match = subst_pattern.match(to)
    
    if fr_match:
        fr_name    = fr_match.group('place').strip()
        fr_country = fr_match.group('country').strip()
    else:
        fr_alt_match = alt_pattern.match(fr)
        if fr_alt_match:
            fr_name = fr_alt_match.group('place')
        
    if to_match:
        to_name    = to_match.group('place').strip()
        to_country = to_match.group('country').strip()
    else:        
        to_alt_match = alt_pattern.match(to)
        if to_alt_match:
            to_name = to_alt_match.group('place')
    
    fr_names.append(fr_name)
    fr_countries.append(fr_country)
    to_names.append(to_name)
    to_countries.append(to_country)

In [18]:
lines['Substation From'] = fr_names
lines['Substation To'] = to_names
lines['Country1'] = fr_countries
lines['Country2'] = to_countries
lines

,Investment number,This investment belongs to project number…,Commissioning Year,"Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction",Type of Element,Substation From,Substation To,Technology,Total route length (km),Country1,Country2
1,4,1,2022,3,ACTransmissionLine,V.Minho,Feira,AC,131.00,NaN,NaN
4,496,4,2022,3,ACTransmissionLine,Fontefria,Vila Nova de Famalicão,AC,140.21,ES,PT
9,60,23,2022,4,ACTransmissionLine,Avelin/Mastaing,Horta,AC,80.00,FR,BE
10,614,26,2023,4,ACTransmissionLine,Nauders,Glorenza,AC,26.00,AT,IT
13,90,33,2023,3,ACTransmissionLine,Calenzano,Colunga,AC,80.00,IT,IT
...,...,...,...,...,...,...,...,...,...,...,...
99,1014,174,2024,3,DCTransmissionLine,Verderio,Bonaduz,DC,165.00,NaN,NaN
139,664,235,2026,3,DCTransmissionLine,"Brunsbüttel, Wilster","Großgartach, Grafenrheinfeld",DC,700.00,NaN,NaN
161,660,254,2024,3,DCTransmissionLine,Osterath,Philippsburg,DC,340.00,DE,DE
203,1458,299,2024,3,DCTransmissionLine,Codrongianos - Lucciana,Suvereto,DC,300.00,NaN,NaN


## create mapping from all unique tyndp substation names to substation names from 'buses'

In [19]:
tyndp_subs   = set(lines['Substation From']).union(set(lines['Substation To']))
tyndp_to_bus = {}

for tyndp in tyndp_subs:
    buses_subs = curated_buses.name.values
    
    closest = min([(bus, Levenshtein.distance(bus.lower(), tyndp.lower())) for bus in buses_subs], key=lambda t: t[1])[0]
    print()
    
    tyndp_to_bus[tyndp] = closest

In [20]:
# a, b = 'Turleenan', 'Guillena'
# a, b = 'Pyhanselka', 'Pyhänselkä'
# a, b = 'Tuomela B', 'Tudela'
# a, b =  'Heviz (HU) \\ Zerjavinec', 'Žerjavinec'
# Levenshtein.distance(a.lower(), b.lower())

In [21]:
# tyndp_to_bus

### Helper functions: Out of all possible pairs of locations from two lists, get the pair whose distance is closest to the specified (line) length
Deals with problem of multiple places in same country sharing a name.

In [22]:
# TODO: keep name!
def extract_coords(rows):
    coordinates = []
    for _, row in rows.iterrows():
        coordinates.append((row['x'], row['y']))
    return coordinates

In [23]:
def match_pair_with_length(s1_rows, s2_rows, length):
    s1_coords = extract_coords(s1_rows)
    s2_coords = extract_coords(s2_rows)
    
    combinations  = list(itertools.product(s1_coords, s2_coords))
    with_distance = [(a, b, geopy.distance.distance(a,b).km) for (a,b) in combinations]
    
    best_match = min(with_distance, key=lambda t: abs(length - t[2]))
    return best_match

# Match start- and endpoints of lines to substations from buses.csv

In [24]:
fr_to_tuples  = {}
# TODO: write out more than indices and use e.g. match for substation 1
error_rows = {}
# TODO: remove this
error_tuples = {}

for index, row in lines.iterrows():    
    # Get closest name match based on Levenshtein distance for start- and endpoints of line
    fr = row['Substation From']
    to = row['Substation To']
    
    fr_country = row['Country1']
    to_country = row['Country2']
            
    s1 = tyndp_to_bus[fr]
    s2 = tyndp_to_bus[to]
    
    # Extract respective rows in buses to determine coordinates
    buses_s1 = curated_buses.loc[curated_buses.name == s1]
    buses_s2 = curated_buses.loc[curated_buses.name == s2]
    
    # If we were able to extract country from name, restrict chosen rows to this country.
    if not pd.isna(fr_country):
        buses_s1 = buses_s1.loc[buses_s1['country'] == fr_country]
    if not pd.isna(to_country):
        buses_s2 = buses_s2.loc[buses_s2['country'] == to_country]
    
    if buses_s1.empty or buses_s2.empty:
        # print(f'{fr}, {fr_country} <-> {to}, {to_country}')
        tpl = (s1, np.NAN, np.NAN, s2, np.NAN, np.NAN, np.NAN)
        error_tuples[index] = tpl
        error_rows[index] = row
        continue
    
    # Choose pair which matches length best
    length = row['Total route length (km)']
    (x1, y1), (x2, y2), coord_dist = match_pair_with_length(buses_s1, buses_s2, length)
        
    tpl = (s1, x1, y1, s2, x2, y2, coord_dist)
    
    # TODO: how to choose an appropriate tolerance?
    if not math.isclose(coord_dist, length, rel_tol=0.45):
        error_tuples[index] = tpl
        error_rows[index] = row
    else:
        fr_to_tuples[index] = tpl

In [25]:
coordinates = pd.DataFrame(index=fr_to_tuples.keys(), data=fr_to_tuples.values(), columns=['s1', 'x1', 'y1', 's2', 'x2', 'y2', 'coord_dist'])

result = lines.copy()
result = result.join(coordinates)

percentage = coordinates.index.size / lines.index.size
print(f'{percentage * 100}% of lines are probably correct.')

# print('Lines where we probably found the correct coordinates:')
# result.loc[~result.s1.isna()]

51.61290322580645% of lines are probably correct.


In [26]:
error_lines = result.loc[result.s1.isna()]
error_subst = set(error_lines['Substation From']).union(error_lines['Substation To'])

# print('')
# {(k,tyndp_to_bus[k]) for k in error_subst}

# Determine coordinates using geopy

In [27]:
def match_pair_with_length_geopy(s1_locations, s2_locations, length):
    s1_first_name = s1_locations[0][0]
    s2_first_name = s2_locations[0][0]

    # Only take locations which at least include name of the first location in list (assumption: best name-based match).
    s1_locations = [l for l in s1_locations if s1_first_name in l[0]]
    s2_locations = [l for l in s2_locations if s2_first_name in l[0]]

    return match_coord_pairs_with_length(s1_locations, s2_locations, length)

In [28]:
def lat_lon(loc):
    return (loc.latitude, loc.longitude)

def match_coord_pairs_with_length(s1_coords, s2_coords, length):
    combinations  = list(itertools.product(s1_coords, s2_coords))
    with_distance = [(a, b, geopy.distance.distance(lat_lon(a),lat_lon(b)).km) for (a,b) in combinations]
    
    best_match = min(with_distance, key=lambda t: abs(length - t[2]))
    return best_match

In [29]:
# locator = Nominatim(user_agent='esm_group')
# geocode = RateLimiter(locator.geocode, min_delay_seconds=0.01)
locator = AlgoliaPlaces(user_agent='esm_group')
geocode = locator.geocode

In [30]:
fr_to_tuples_geopy = {}
error_tuples_geopy = {}

for index, row in error_lines.iterrows():
    fr   = row['Substation From']
    to   = row['Substation To']
    dist = row['Total route length (km)']

    fr_country = row['Country1']
    to_country = row['Country2']

    # TODO: is it possible to get several matching locations?
    fr_locs = geocode(fr, exactly_one=False) if pd.isna(fr_country) else geocode(fr, exactly_one=False, countries=[fr_country])
    to_locs = geocode(to, exactly_one=False) if pd.isna(to_country) else geocode(to, exactly_one=False, countries=[to_country])
    
    if fr_locs is None or to_locs is None:
        tpl = (fr, np.NAN, np.NAN, to, np.NAN, np.NAN, np.NAN)
        error_tuples[index] = tpl
        continue
        
    (s1, (x1, y1)), (s2, (x2, y2)), coord_dist = match_pair_with_length_geopy(fr_locs, to_locs, dist)
    tpl = (s1, x1, y1, s2, x2, y2, coord_dist)

    if not math.isclose(coord_dist, dist, rel_tol=0.45):
        error_tuples_geopy[index] = tpl
    else:
        fr_to_tuples_geopy[index] = tpl

In [31]:
coordinates_geopy = pd.DataFrame(index=fr_to_tuples_geopy.keys(), data=fr_to_tuples_geopy.values(), columns=['s1', 'x1', 'y1', 's2', 'x2', 'y2', 'coord_dist'])

coordinates = coordinates.append(coordinates_geopy)

result = lines.copy()
result = result.join(coordinates)

percentage = coordinates.index.size / lines.index.size
print(f'{percentage * 100}% of lines are probably correct.')

print('Lines where we probably found the correct coordinates:')
result.loc[~result.s1.isna()]

73.11827956989248% of lines are probably correct.
Lines where we probably found the correct coordinates:


,Investment number,This investment belongs to project number…,Commissioning Year,"Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction",Type of Element,Substation From,Substation To,Technology,Total route length (km),Country1,Country2,s1,x1,y1,s2,x2,y2,coord_dist
1,4,1,2022,3,ACTransmissionLine,V.Minho,Feira,AC,131.00,NaN,NaN,V. Minho,-7.761841,41.582580,Feira,-8.368835,40.953974,96.475112
4,496,4,2022,3,ACTransmissionLine,Fontefria,Vila Nova de Famalicão,AC,140.21,ES,PT,Fontefría,42.476200,-7.789700,Vila Nova de Famalicão,41.407900,-8.519800,133.210461
9,60,23,2022,4,ACTransmissionLine,Avelin/Mastaing,Horta,AC,80.00,FR,BE,Mastaing,3.353577,50.316531,Horta,3.574677,51.015483,81.423032
10,614,26,2023,4,ACTransmissionLine,Nauders,Glorenza,AC,26.00,AT,IT,Nauders,46.891700,10.502600,Glurns - Glorenza,46.671400,10.553900,24.801352
13,90,33,2023,3,ACTransmissionLine,Calenzano,Colunga,AC,80.00,IT,IT,Calenzano,11.126404,43.871168,Colunga,11.284332,44.494546,70.285438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,661,132,2025,3,DCTransmissionLine,Emden East,Osterath,DC,300.00,DE,DE,Elsterstraße,53.365000,7.154900,Osterather Straße,50.960100,6.936370,268.011319
99,1014,174,2024,3,DCTransmissionLine,Verderio,Bonaduz,DC,165.00,NaN,NaN,Verderio,9.342499,45.755068,Bonaduz,9.253235,46.824496,117.908376
139,664,235,2026,3,DCTransmissionLine,"Brunsbüttel, Wilster","Großgartach, Grafenrheinfeld",DC,700.00,NaN,NaN,Brunsbüttel,9.242249,53.906765,Grafenrheinfeld,10.081329,49.950336,444.022075
161,660,254,2024,3,DCTransmissionLine,Osterath,Philippsburg,DC,340.00,DE,DE,Osterath,6.586304,51.245584,Philippsburg,8.411407,49.204140,302.500199


In [32]:
error_coordinates_geopy = pd.DataFrame(index=error_tuples_geopy.keys(), data=error_tuples_geopy.values(), columns=['s1', 'x1', 'y1', 's2', 'x2', 'y2', 'coord_dist'])


In [33]:
error_coordinates_geopy.join(lines)

,s1,x1,y1,s2,x2,y2,coord_dist,Investment number,This investment belongs to project number…,Commissioning Year,"Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction",Type of Element,Substation From,Substation To,Technology,Total route length (km),Country1,Country2
24,Gabčíkovo,47.89510,17.57810,Gönyű,47.7331,17.82790,25.969975,1500,48,2020,4,ACTransmissionLine,Gabcikovo,Gonyu,AC,110.0,SK,HU
28,Riga,56.94600,24.10590,Riga,56.9460,24.10590,0.000000,1062,62,2020,4,ACTransmissionLine,Riga CHP2,Riga HPP,AC,15.0,LV,LV
45,Keminmaa,65.80300,24.52090,Messure Crescent,43.4910,-79.88450,6286.587625,396,111,2025,3,ACTransmissionLine,Keminmaa,Messaure,AC,200.0,NaN,NaN
72,Vilnius kalea/Calle Vilnius,42.84500,-2.65328,Néris-les-Bains,46.2879,2.66137,569.512881,382,170,2025,3,ACTransmissionLine,Vilnius,Neris,AC,80.0,NaN,NaN
76,Bălți,47.76310,27.92930,Tartupis,54.5966,23.83800,811.934190,1012,170,2023,3,ACTransmissionLine,Balti,Tartu,AC,168.0,NaN,NaN
77,Virú,-8.41429,-78.75220,Tsirguliina,57.8640,26.19470,11677.913915,1013,170,2025,3,ACTransmissionLine,Viru,Tsirguliina,AC,243.0,NaN,NaN
80,Germanto ežero Lietuvos 1000-čio parko special...,55.98280,22.16250,Vilnius,55.9301,25.11200,184.313772,1565,170,2020,6,ACTransmissionLine,Lietuvos E,Vilnius,AC,41.0,NaN,NaN
86,Bitėnai,55.15180,25.38030,Nong Khae,14.3406,100.86700,7788.976319,1634,170,2025,3,ACTransmissionLine,Bitenai,KHAE,AC,234.0,NaN,NaN
103,Güterweg Seyring,48.23870,13.44550,Zayante,37.0919,-122.04400,9578.538532,886,186,2022,4,ACTransmissionLine,Seyring,Zaya,AC,60.0,NaN,NaN
201,Liefkenshoek,51.13650,4.75960,Mercatorstraße,50.2902,6.08770,132.895031,604,297,2025,3,ACTransmissionLine,Liefkenshoek,Mercator,AC,19.0,BE,BE


In [34]:
# TODO: plot error tuples and matches separately.